In [ ]:
!pip install ultralytics roboflow

import os
from ultralytics import YOLO
from roboflow import Roboflow
import yaml
from pathlib import Path

In [ ]:
!pip install torch torchvision torchaudio

In [3]:
import torch

In [ ]:
rf = Roboflow(api_key="1VHz8QF5F69ddtGCCfTg")
project = rf.workspace("sml-project-hfi0w").project("footprint-classification")
dataset = project.version(1).download("folder")

In [ ]:
# Initialize YOLOv8 classification model
model = YOLO('yolov8n-cls.pt')

In [6]:
# Training configuration
training_config = {
    'data': 'footprint-classification-1',  # Path to dataset
    'epochs': 100,
    'imgsz': 640,
    'batch': 16,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 8,
    'patience': 50,  # Early stopping patience
    'save': True,  # Save results
    'project': 'footprint_classification',
    'name': 'yolov8_classification',
    'pretrained': True,  # Use pretrained weights
    'optimizer': 'Adam',  # Using Adam optimizer
    'lr0': 0.001,  # Initial learning rate
    'lrf': 0.01,  # Final learning rate fraction
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'verbose': True,
    'seed': 42
}

In [ ]:
# Install required packages
!pip install ultralytics roboflow

from ultralytics import YOLO
from roboflow import Roboflow
import os
import torch
import shutil
from pathlib import Path

# Initialize Roboflow and download dataset
rf = Roboflow(api_key="1VHz8QF5F69ddtGCCfTg")
project = rf.workspace("sml-project-hfi0w").project("footprint-classification")
dataset = project.version(1).download("folder")

# Get the actual dataset path from the Roboflow dataset object
base_dataset_path = dataset.location  # This gets the actual path where Roboflow downloaded the data


In [ ]:

# Function to organize dataset into YOLOv8 classification format
def organize_dataset(base_path):
    # Create main directories
    dataset_root = Path('dataset')
    if dataset_root.exists():
        shutil.rmtree(dataset_root)

    # Source directory from Roboflow download
    source_path = Path(base_path)

    # Create and populate train, valid, test directories
    for split in ['train', 'valid', 'test']:
        split_source = source_path / split
        if split_source.exists():
            split_dest = dataset_root / split
            split_dest.mkdir(parents=True, exist_ok=True)

            # Copy all class folders
            if split_source.is_dir():
                for item in split_source.iterdir():
                    if item.is_dir():
                        shutil.copytree(item, split_dest / item.name)

    return str(dataset_root.absolute())

# Print the base dataset path
print(f"Downloaded dataset location: {base_dataset_path}")

# Organize dataset
dataset_path = organize_dataset(base_dataset_path)
print(f"Organized dataset at: {dataset_path}")


In [ ]:

# Print dataset structure to verify
def print_dataset_structure(path):
    path = Path(path)
    print("\nDataset structure:")
    total_images = 0
    for split in ['train', 'valid', 'test']:
        split_path = path / split
        if split_path.exists():
            print(f"\n{split} folder:")
            split_total = 0
            for class_folder in split_path.iterdir():
                if class_folder.is_dir():
                    n_images = len(list(class_folder.glob('*')))
                    print(f"  {class_folder.name}: {n_images} images")
                    split_total += n_images
            print(f"  Total {split} images: {split_total}")
            total_images += split_total
    print(f"\nTotal dataset images: {total_images}")

print_dataset_structure(dataset_path)


In [ ]:

# Initialize YOLOv8 classification model
model = YOLO('yolov8n-cls.pt')

# Training configuration
training_config = {
    'data': dataset_path,
    'epochs': 100,
    'imgsz': 640,
    'batch': 16,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 8,
    'patience': 50,
    'save': True,
    'project': 'footprint_classification',
    'name': 'yolov8_classification',
    'pretrained': True,
    'optimizer': 'Adam',
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'verbose': True,
    'seed': 42
}


In [ ]:

# Start training
results = model.train(**training_config)

# Evaluate the model on validation set
val_results = model.val()

# Export the model to ONNX format (optional)
model.export(format='onnx')

# Get the path to the best weights
best_weights_path = Path(f"{training_config['project']}/{training_config['name']}/weights/best.pt")


In [ ]:

def test_model(model_path, image_path):
    model = YOLO(model_path)
    results = model.predict(image_path, task='classify')

    print(f"Predictions for {image_path}:")
    for r in results:
        probs = r.probs
        top_k = probs.top5
        for i, (cls_idx, prob) in enumerate(zip(top_k[0], top_k[1])):
            print(f"#{i+1} Class: {r.names[cls_idx]} | Probability: {prob:.4f}")


In [ ]:

print(f"\nTraining completed! Best weights saved at: {best_weights_path}")
print("\nModel performance metrics:")
print(f"Top-1 Accuracy: {val_results.top1}")
print(f"Top-5 Accuracy: {val_results.top5}")